当给一个pandas 对象赋值时, 一定要小心, 避免使用 `Chained Indexing`进行赋值, 而应该使用 `.loc[:,'column']` 方式赋值.

In [1]:
import pandas as pd
import numpy as np

#### 索引

我们先用 Chained Index 和 loc Index分别去索引一个Dataframe中的一列.

In [2]:
raw_list = [list('abcd'),list('efgh'),list('ijkl'),list('mnop')]
dfmi = pd.DataFrame(raw_list, columns=pd.MultiIndex.from_product([['one', 'two'], ['first', 'second']]))

In [3]:
dfmi

one          two       
  first second first second
0     a      b     c      d
1     e      f     g      h
2     i      j     k      l
3     m      n     o      p

**在以上的代码中, `MultiIndex` 是模拟 `Hierarchical Indexing`, 这种Index方式允许程序员以低维度数据结构(Series, DataFrame)存储和操作任意维度的数据. MultiIndex 类可以通过 List of Arrays, List of Tuples,和两个list的笛卡尔积去创建.**

现在如果要索引到 ('one', 'second') 这一列, 有两种索引方法:

1：Chained 方式

In [6]:
dfmi['one']['second']

0    b
1    f
2    j
3    n
Name: second, dtype: object

2: .loc 方式

In [7]:
dfmi.loc[:, ('one', 'second')]

0    b
1    f
2    j
3    n
Name: (one, second), dtype: object

第一种Chained方式, 等价于 `dfmi.__get__item__('one').__get__item__('second') `.

即会先用 `dfmi['one']` 索引到一个DataFrame, 我们把它命名为dfmi_one (结果见上), 然后用 `dfmi_one['second']` 去索引Series. Pandas在这个过程中调用了两次 `__get_item__` 方法.

第二种.loc 方式, `dfmi.loc[:, ('one', 'second')]`等价于 `dfmi.loc.__get_item__((slice(None), ('one', 'second')))`.

将一组嵌套tuple `(slice(None), ('one', 'second'))` 直接传递给 `__get_item__`, 只用一次操作就将期望的Series取出. 这种方式明显速度更快.

当然这只是速度上的对比, 当我们用以上两种方式赋值时,会发生什么?

#### 不同的赋值方式与SettingWithCopyWarning

In [39]:
to_set_value_1 = pd.Series(list('xyza'))
dfmi_chained = pd.DataFrame(raw_list, columns=pd.MultiIndex.from_product([['one', 'two'], ['first', 'second']]))
dfmi_loc = pd.DataFrame(raw_list, columns=pd.MultiIndex.from_product([['one', 'two'], ['first', 'second']]))

在上面的代码中, 我们新建一个Series. 以及两个相同的DataFrame, 后续会分别用于chained索引并赋值, 以及.loc索引并赋值.

In [41]:
dfmi_chained['one']['second'] = to_set_value

/Users/zxwang/.pyenv/versions/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [42]:
dfmi_chained

one          two       
  first second first second
0     a      x     c      d
1     e      y     g      h
2     i      z     k      l
3     m      a     o      p

In [44]:
dfmi_loc.loc[:, ('one', 'second')] = to_set_value

In [45]:
dfmi_loc

one          two       
  first second first second
0     a      x     c      d
1     e      y     g      h
2     i      z     k      l
3     m      a     o      p

先来看一下上面的代码发生了什么. 

两种不同的赋值方式得到的结果如下:

1. Chained 赋值:
我们在赋值结束后会看到一个 Warning:"SettingWithCopyWarning". 查看 `dfmi_chained` 的值,此时对应的索引已经赋值. 

2. .loc 赋值:
正常赋值, 没有Warning, 查看 `dfmi_loc` 的值, DataFrame中对应的值已经赋值成功.

那什么是 "SettingWithCopyWarning"? 为什么Chained Indexing 赋值会引发这个Warning?

"SettingWithCopyWarning" warning指你可能在给一个被Copy的Datframe赋值, 这个赋值很可能不会影响到原Dataframe.

我们把 `dfmi_chained['one']['second'] = to_set_value` 这个赋值语句等价于以下语句: 

    `dfmi.__get_item__('one').__set_item_('second', to_set_value)`

注意它会先使用 `__get_item__('one')` 拿出一个Dataframe. 在其它情况下可能是拿出一个Series, 或者其它什么对象. Pandas此时并不清楚拿到的对象, 是原对象的引用, 还是一个被Copy过的新对象. 因此也就无法保证 `__set_item('second', to_set_value)` 这个赋值语句, 会对原来的对象也赋值. 这也是Warning被抛出的原因.

对于第二种 .loc 索引 + 赋值的语句 `dfmi.loc[:, ('one', 'second')] = to_set_value`. 等价于以下语句
    `dfmi.loc.__set_item__((slice(None), ('one', 'second')), to_set_value)`
嗯..所以是直接调用了 .loc属性返回值的 `__set_item__` 方法. Pandas已经确保了 `dataframe.loc`属性就是 dataframe 自身, 只不过是更改了dataframe的索引方法, 因此 `.loc.__set_item__()` 就是对dataframe自身进行操作的. 

也可以理解为, 第二种方式,并不是先“索引”Series，再赋值. 而是直接将值赋给dataframe里对应索引的Series. 

如果第二种方式赋值方法等价于
 `dfmi.loc.__get__item__(params).__set_item__(to_set_value)`,  那么其实也是会遇到 "SettingWithCopyWarning", 因为 `__get_item__()` 方法有可能返回对象的引用, 有可能返回Copy值. 

所以两者的区别在于 Chained 赋值的方式是先索引到对象, 再更改对象的值. 而 .loc 赋值是直接去更改原始Dataframe的对应索引的值.
